In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import torch

SEQ_LEN = 50
NUM_FEATURES = 4 * 25  # 25 landmarks X 4 features each - x,y,z,vis


def read_raw_data(score_path, data_path):
    shot_names = {}

    score_df = pd.read_csv(score_path)
    score_df = score_df[score_df['frames'] < SEQ_LEN]

    data_df = pd.read_csv(data_path)
    amount_of_shots = len(score_df)
    print('all examples:', amount_of_shots)

    single_shot_x = np.zeros((SEQ_LEN, NUM_FEATURES))
    x = np.zeros((amount_of_shots, SEQ_LEN, NUM_FEATURES))
    y = []
    shot_index = 0

    for _, row in tqdm(score_df.iterrows()):

        frames = row['frames']
        score = row['score']
        shot_name = row['name']

        for i in range(0, frames):
            frame_name = shot_name + '_{}'.format(i)
            shot_data = []
            for k, v in data_df[data_df['name'] == frame_name].iteritems():
                if k == 'name':
                    continue
                shot_data.append(v.values[0])
            single_shot_x[i] = np.array(shot_data)

        if shot_index == amount_of_shots:
            break

        y.append(score)
        x[shot_index] = np.nan_to_num(single_shot_x)
        shot_names[shot_index] = shot_name
        shot_index += 1
    return torch.from_numpy(x).float(), np.array(y), shot_names


# keep x and y unchanged
x, y, names = read_raw_data(score_path='labels/fts_score.csv', data_path='labels/fts_data.csv')

all examples: 1371


0it [00:00, ?it/s]

visualization

In [ ]:
from visualization import points_visualization

points_visualization.create_gif_for_given_shot(names[1],x[1])

feature selection

In [12]:

def feature_selection(remove_features):
    
    num_of_landmark_to_remove = len(remove_features)*4
    copy = x.clone() # don't change x
    
    SEQ_LEN = 50
    NUM_OF_POINTS=25
    ALL_EXISTING_FEATURES = 4 * NUM_OF_POINTS
    NUM_FEATURES = 4 * 25 - num_of_landmark_to_remove 
    amount_of_shots= len(x)
    single_shot_x = np.zeros((SEQ_LEN, NUM_FEATURES))
    new_frame_after_removing_features=np.array([])
    shot_index = 0
    seq_index=0
    copy.numpy()
    new_frame = np.zeros((amount_of_shots, SEQ_LEN, NUM_FEATURES))
    for shot in copy:
        shot_data = []
        for frame in shot:
            new_frame_after_removing_features=np.array([])
            for point_num in range(0,NUM_OF_POINTS):
                if point_num not in remove_features:
                    point_features = frame[point_num*4:point_num*4+4]
                    new_frame_after_removing_features = np.concatenate((new_frame_after_removing_features,point_features))
            if seq_index == SEQ_LEN-1:
                seq_index=0
                break
            single_shot_x[seq_index] = np.array(new_frame_after_removing_features) 
            seq_index+=1
        shot_index += 1
        if shot_index == amount_of_shots-1:
         break
        new_frame[shot_index] = np.nan_to_num(single_shot_x)


    return torch.from_numpy(new_frame).float()

#Usage example - remove head landmarks 
landmarks_to_remove = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
data = feature_selection(landmarks_to_remove)
print(data.shape)



torch.Size([1371, 50, 56])
